# <center> get evolving temperature for academic paper

In [1]:
import networkx as nx
import pandas as pd
from scipy import sparse
import numpy as np
from time import perf_counter
from sklearn.preprocessing import MinMaxScaler
from itertools import product, combinations, chain
from copy import deepcopy

## domain

In [2]:
class Domain:
    '''
    domain with one pionneering paper
    '''
    def __init__(self):
        self.G = nx.DiGraph()
        self.pioneer_node = 0  # pionneering node id in G, not the original one
        self.num_of_nodes = 0 
        self.num_of_edges = 0
        
        self.skeleton_tree = nx.DiGraph()
        self.node_node_reduction_index = None
        self.node_network_reduction_index = None
        self.node_node_average_step = 0
        
        self.average_temperature = 0
        self.average_growth_temperature = 0
        self.average_struct_temperature = 0
        self.volume = 0  # int
        self.mass = 0  # float
        self.node_tree_entropy = None
        self.node_temperature = None
        self.evolving_until_yr = 0
        self.birth_yr = 0
        
        self.von_neumann_entropy = 0
        
    def setBirthYear(self, yr):
        self.birth_yr = yr
        
    def evolve(self, df, lead_paper_id, start_yr, step=1):
        # extract reference for a year
        df_t = df[(df['year']>= start_yr) & (df['year']< start_yr+step)][['paper_id','reference_id']]
        G_t = nx.from_pandas_edgelist(df_t, source = 'reference_id', target = 'paper_id', create_using = nx.DiGraph)
        self.G.add_nodes_from(G_t.nodes)
        self.G.add_edges_from(G_t.edges)
        
        # add self-loop for pioneer paper (if no self-loop yet in G)
        self.G.add_edge(lead_paper_id, lead_paper_id)

        # initialize class attributes
        self.pioneer_node = np.argwhere(np.array(self.G.nodes)==lead_paper_id).flatten()[0]
        self.num_of_nodes = self.G.number_of_nodes()
        self.num_of_edges = self.G.number_of_edges()
        self.node_node_reduction_index = np.zeros((self.num_of_nodes, self.num_of_nodes))
        self.node_network_reduction_index = np.empty(self.num_of_nodes)
        self.node_tree_entropy = np.empty(self.num_of_nodes)
        self.node_temperature = np.empty(self.num_of_nodes)
        self.evolving_until_yr = min(start_yr+step-1, 2020)
        
        # clear last round's weight attribute
        for (n1, n2, d) in self.G.edges(data=True):
            d.clear()
        
        self.von_neumann_entropy = 0
        self.average_temperature = 0
        self.average_growth_temperature = 0
        self.average_struct_temperature = 0
        
    def getVonNeumannEntropy(self):
        '''
        Compute G's von neumann entropy 
        '''
        for c in sorted(nx.strongly_connected_components(self.G), key=len, reverse=True):
            sc_node_num = len(c)
            if sc_node_num>1:
                sub_G = self.G.subgraph(c).copy()
                # add weight attr 
                nx.set_edge_attributes(sub_G, 1,'weight')
                # get res
                res = sum([wgt**2 * sub_G.in_degree(u)/sub_G.in_degree(v)/sub_G.out_degree(u)**2 \
                           for (u,v,wgt) in sub_G.edges.data('weight')])  #combinations(c, 2):   
                self.von_neumann_entropy += 1-1/sc_node_num - res/2/sc_node_num**2
            else:
                # strongly connected component containing 1 node
                # von neumann entropy = 0
                break
        print("von neumann entropy")
        print(self.von_neumann_entropy)    
    
    def setNodeReductionIdx(self):
        '''
        Compute node-node and node-network reduction index
        Output:
            Reduction_idx: a 2-D array of size (# node number * # node number)
        '''
        node_embeddings = self.getNodeEmbedding()
        # node distance in embedding space
        sparse_distance_mtrx = sparse.csr_matrix(self.getNodeDistance(node_embeddings.A))  # matrix.A  convert from matrix to ndarray
        # weighted sparse adj matrix 
        sparse_adjacency_mtrx  = nx.to_scipy_sparse_matrix(self.G)
        sparse_weighted_adj_mtrx = sparse_adjacency_mtrx.multiply(sparse_distance_mtrx) # element wise multiplication
        # maximal node-pair distance in embedding space
        max_distance = sparse_weighted_adj_mtrx.max()
        # build weighted graph from sparse adj matrix
        weighted_G = nx.from_scipy_sparse_matrix(sparse_weighted_adj_mtrx, create_using = nx.DiGraph, edge_attribute='weight')
        # average node pair shortest distance 
        shortest_paths = dict(nx.all_pairs_dijkstra_path(weighted_G, weight='weight'))
        self.node_node_average_step = self.getAvgStep(shortest_paths)
        
        # node-node reduction index
        wgts = nx.get_edge_attributes(weighted_G, 'weight')
        for i,j in product(range(self.num_of_nodes),range(self.num_of_nodes)):  # i --> j, find j's predecessors
            if i==j:
                continue 
            # get all shortest paths from i to j's references     
            paths = list([shortest_paths[i].get(pre_j, []) for pre_j in weighted_G.predecessors(j)])
            for p in paths:
                if not p:  # if p is an empty list
                    self.node_node_reduction_index[i,j] += max_distance * self.node_node_average_step
                if len(p)>1:  # len(p)==1, path to itself, self-loop, no reduction     
                    self.node_node_reduction_index[i,j] += np.sum([wgts[(p[i],p[i+1])] for i in range(len(p)-1)]) 
        
        # add edge weight
        mapping = {i:u for i,u in enumerate(self.G.nodes)}
        nx.relabel_nodes(weighted_G, mapping, copy = False)
        self.G = weighted_G
        
        # node-network reduction index
        self.node_network_reduction_index = self.node_node_reduction_index.T @ np.ones((self.num_of_nodes,1))
    
    def buildSkeletonTree(self):
        '''
        build domain skeleton tree by trimming domain graph
       
        '''
        # skeleton tree
        T = deepcopy(self.G)
        
        ### Step 1
        # detect loops in T
        # cut an edge whose extremities have the biggest difference in reduction index 
        # keep graph connectivity 
        for i, src in enumerate(list(T.nodes)):
            for target in list(T.nodes)[i+1:]: # src != target
                if nx.has_path(T, src, target) and nx.has_path(T, target, src):
                    has_loop = True
                    while has_loop:
                        try:
                            loop = nx.shortest_path(T, source=src, target=target)[:-1] + \
                                   nx.shortest_path(T, source=target, target=src)
                            self.cutEdgeInLoop(T, loop)
                        except:
                            has_loop = False

        ### Step 2
        # leave only one predecessor/reference for each paper
        # remove directed edges

        domain_nodes = T.nodes
        for i,n in enumerate(domain_nodes):
            pre_n = list(T.predecessors(n))
            pre_n_idx = [np.argwhere(np.array(domain_nodes)==j).flatten()[0] for j in pre_n]
            self_network_reduction_idx = self.node_network_reduction_index[i]
            if len(pre_n)>1:
                delta_reduction_idx = np.array([abs(self.node_network_reduction_index[j] -
                                                    self_network_reduction_idx) for j in pre_n_idx])
                reference_to_keep = pre_n.pop(np.argmin(delta_reduction_idx))      
                for src in pre_n:
                    T.remove_edge(src, n)          
                    
        self.skeleton_tree = T   
        
    def setNodeTreeEntropy(self):
        '''
        Compute tree entropy based on domain skeleton tree
        Input:
            T: nx.graph.DiGraph, skeleton tree
        Output:
            T: nx.graph.DiGraph, same skeleton tree with tree entropy added as a node attribute
        '''
        T = self.skeleton_tree
        m = T.number_of_edges()
        node_set = set(T.nodes)
    
        for i,n in enumerate(T.nodes):
            if i==self.pioneer_node:
              #  T.nodes[n]['tree_entropy'] = 0  # root entropy is always 0
                self.node_tree_entropy[i] = 0
            else:    
                A_set = self.getSubTreeNodes([n])
             #   B_set = node_set.difference(A_set)
             #   cut_size = nx.cut_size(T, A_set, B_set) + nx.cut_size(T, B_set, A_set)
                n_parents = list(T.predecessors(n))
                sub_parent_T_nodes  = self.getSubTreeNodes(n_parents)
                sub_T_volume = max(nx.volume(T, list(A_set), weight = None),1)  # sub_T_nodes
            #    sub_parent_T_volume = max(nx.volume(T, list(sub_parent_T_nodes), weight = None),1)
            #    self.node_tree_entropy[i] = -cut_size/(2*m)*np.log(sub_T_volume/sub_parent_T_volume)
                if sub_parent_T_nodes:
                    self.node_tree_entropy[i] = -sub_T_volume/(2*m)*np.log(len(A_set)/len(sub_parent_T_nodes))
                else:
                    self.node_tree_entropy[i] = 0  # isolated node
                
    def setStructTemperature(self, T):
        self.average_struct_temperature = T
    
    def getAvgTemperature(self, is_initial = True, is_big_graph = True, old_avg_temperature = 0, old_V = 0, old_N = 0):
        '''
        Compute volume, mass and average temperature based on domain skeleton tree
        Input: 
            is_initial: boolean, whether time stamp = 0
            old_avg_temperature: float, average growth temperature for the last time stamp
            old_V: int, volume of the last time stamp
            old_N: float, mass of the last time stamp       
        '''
        ###### get average growth temperature 
        Tree = self.skeleton_tree
        # volume
        self.volume = self.num_of_nodes
        # rescale node-node reduction index
        scaler = MinMaxScaler()
        scaled_node_reduction_index = scaler.fit_transform(self.node_node_reduction_index)
        # mass
        tree_adj = nx.to_scipy_sparse_matrix(Tree).todense()
        tree_adj = np.where(tree_adj == 0,tree_adj, 1)
        self.mass = self.num_of_nodes - np.sum(np.multiply(scaled_node_reduction_index, tree_adj))
        # scaling coef
        coef = 10 if is_big_graph else 100
        if is_initial:
            # entropy 
            S = np.sum(self.node_tree_entropy)
            # set default constant
            R = 8
            c = 1 
            self.average_growth_temperature =  coef * np.exp(S/(c*self.mass)) * np.float_power(self.mass/self.volume,R/c)
        else:
            self.average_growth_temperature = old_avg_temperature * (self.volume/old_V) * (old_N/self.mass)
        print("volume:{}, mass:{}, T_growth:{}".format(self.volume, self.mass, self.average_growth_temperature))  
        
        ###### get average temperature
        self.average_temperature = self.average_growth_temperature + abs(self.average_struct_temperature)
            
    
    def spreadTemperature(self, former_population = 0):
        # get inactive nodes 
        coldest_nodes = self.getInactiveNodes(former_population)
            
        # add distance attribute to domain reference graph
        scaler = MinMaxScaler(feature_range = (0.5,1.5))
        scaled_node_node_reduction_idx = scaler.fit_transform(self.node_node_reduction_index)
        node_list = list(self.G.nodes)
        for u,v in self.G.edges:
            self.G.edges[u,v]['distance'] = scaled_node_node_reduction_idx[node_list.index(u)][node_list.index(v)] 

        # get node temperature     
        self.node_temperature = self.diffuseHeat(self.pioneer_node, coldest_nodes, int(self.node_node_average_step))   
        
                        
    ########## auxiliary functions ##########
    def getNodeEmbedding(self):

        sparse_adjacency_mtrx  = nx.to_scipy_sparse_matrix(self.G)
        sparse_degree_mtrx = np.diagflat(sparse_adjacency_mtrx.T @ np.ones((sparse_adjacency_mtrx.shape[0],1), dtype=int))
        # Laplacian matrix 
        L = sparse_degree_mtrx - sparse_adjacency_mtrx.T
        # regular Laplacian matrix
        L_regular = np.sqrt(sparse_degree_mtrx) @ L @ np.sqrt(sparse_degree_mtrx)       
        # node embedding (projected dimension = # node number)
        _, eigvecs = np.linalg.eig(L_regular)
        return eigvecs
    
    def getNodeDistance(self, M):
        '''
        Create distance matrix in embedding space 
        Extract node-pair distance for the graph
        Input: 
            M: 2-D ndarray, eigenvectors of the regular Laplacian matrix, each row is a vector
        Output:
            D: 2-D ndarray, distance matrix of the graph
        '''
        N = M.shape[0]
        D = np.empty(M.shape)
        for i in range(N):
            for j in range(i, N):
                d = np.linalg.norm(M[i] - M[j]) # 2-norm
                D[i,j], D[j,i] = d, d
        return D 
                        
    def getAvgStep(self, paths):
        '''
        Input: 
            paths: A dictionary of shortest paths dictionary
        Output: 
            average shortest path length
        '''
        total_step = 0
        path_counter = 0                        
        for path_dict in paths.values():
            steps = [len(p)-1 for p in path_dict.values()]
            total_step += np.sum(steps)
            path_counter += len(path_dict)-1
        return total_step/path_counter               
    
    def cutEdgeInLoop(self, T, loop):
        '''
        cut an edge in a loop in domain skeleton tree T
        Input:
            T: networkx.graph.DiGraph  unweighted/weighted directed graph
            loop: list representing a loop
        Output:
            None. T is already modified (dict object is changeable) 
        '''
        nodes = np.array(T.nodes)
        is_cut = False
        edges_to_cut = dict([((loop[i], loop[i+1]), 
                              abs(self.node_network_reduction_index[i+1] - self.node_network_reduction_index[i])) 
                             for i in range(len(loop)-1)])
        edges_to_cut = sorted(edges_to_cut.items(), key = lambda x: x[1], reverse=True) # sort by network reduction index 
        for edge,val in edges_to_cut:
            if len(list(T.predecessors(edge[1]))) > 1:
                T.remove_edge(edge[0], edge[1])
                is_cut = True
        if not is_cut:
            T.remove_edge(edges_to_cut[0][0][0], edges_to_cut[0][0][1])
                    
    def getSubTreeNodes(self, parents):
        '''
        Find all the children of node n in domain skeleton tree, BFS search
        Input:
            parents: list of int, root nodes' ids
        Output:
            sub_T_nodes: set of int, all child nodes of parents and parents themselves
        '''
        sub_T_nodes = set()
        child_nodes_to_parse = parents
        while child_nodes_to_parse:
            p = child_nodes_to_parse.pop(0)
            sub_T_nodes.add(p)
            child_nodes_to_parse.extend(list(self.skeleton_tree.successors(p))) # p has direct children
        return sub_T_nodes
    
    def getInactiveNodes(self, former_population = 0):
        '''
        Input:
            former_population: int, node number by the end of last time stamp
        Output:
            inactive_leaves: list, inactive node ids
        '''
        inactive_leaves = list()
        for i,n in enumerate(self.G.nodes):
            children = list(self.G.successors(n))
            if not children:
                inactive_leaves.append(i) 
            elif i < former_population and max(children)<former_population:
                inactive_leaves.append(i)
        return inactive_leaves
    
    def diffuseHeat(self, hottest_nodes, coldest_nodes, num_of_round):
        #   diffuseHeat(self.pioneer_node, coldest_nodes, N = int(self.node_node_average_step))
        '''
        Compute heat diffusion over a skeleton tree
        Input:
            G: nx.graph.DiGraph,  loopless graph with node attribute 'tree_entropy'
            hottest_nodes: list of int, nodes at temperature 1
            coldest_nodes: list of int, nodes at temperature 0
            num_of_round: int, number of steps    
        Output: 
            Temperatures: ndarray, node temperature list after N steps 
        '''    
       
        adjacency = nx.to_scipy_sparse_matrix(self.G, weight = 'distance').T   
        n = adjacency.shape[0]
        Temperatures = 0.5 * np.ones(n, float)
        sources = np.array(hottest_nodes)  
        targets = np.array(coldest_nodes)

        # fix source and target temperature
        Temperatures[sources] = 1 # red
        Temperatures[targets] = 0  # blue

        # compute transition matrix
        wgts = adjacency.dot(np.ones(n))
        inverse_weight_matrix = sparse.diags(1 / wgts, format='csr')
        P = inverse_weight_matrix.dot(adjacency)

        
        # heat diffusion 
        Temperatures = P.T.dot(Temperatures)
        Temperatures = (Temperatures - min(Temperatures))/(max(Temperatures) - min(Temperatures))
        Temperatures[sources] = 1   
        Temperatures[targets] = 0
        for i in range(num_of_round):
            Temperatures = P.dot(Temperatures)   
            # fix source and target temperature
            Temperatures[sources] = 1   
            Temperatures[targets] = 0    

        # rescale temperature    
        mean_T = np.mean(Temperatures)    
        Temperatures *= self.average_temperature/mean_T    
        return Temperatures
    
  
    

### $T_{struct}^t$ ---- shrink $G^{t+1}$

In [3]:
def get_von_neumann_E_shrinked(prev_G, G, node_df):
    '''
    shrink G to a graph induced by nodes in prev_G
    get von neumann entropy on the shrinked graph
    Input:
        prev_G: nx.DiGraph
        G: nx.DiGraph
        node_df: pandas DataFrame, columns = ['paper_id','year']
    Output:
        E: von neumann entropy of the graph shrinked from G
    '''
    
    if not prev_G.number_of_nodes():
        return 0,0
    
    node_df = node_df.set_index('paper_id')
    new_edge_wgt_sum = 0
    # shrink network
    nx.set_edge_attributes(G, 1,'weight')
    nodes_to_rm = set(G.nodes).difference(set(prev_G.nodes))
    for n in nodes_to_rm:
        parents_n = set(G.predecessors(n))
        if len(parents_n) == 1:
            for c in G.successors(n):
                G.add_edge(next(iter(parents_n)), c, weight = G[n][c]['weight']/2) 
            G.remove_node(n)
        else:
            # more than 1 parent
            # find all youngest ancestor nodes in prev_G
            # add virtual edges between them, weight = 1/# youngest ancestor nodes
            ancestors_n = parents_n
            wgt_shared = sum([G[p][n]['weight'] for p in ancestors_n])
            tmp = ancestors_n.intersection(nodes_to_rm)
            while tmp:
                tmp_ancestors = []
                tmp_ancestors.extend([list(G.predecessors(k)) for k in tmp])
                tmp_ancestors = list(chain.from_iterable(tmp_ancestors))  # unlist nested list
                ancestors_n.union(set(tuple(tmp_ancestors)))
                ancestors_n -= tmp # set difference
                tmp = ancestors_n.intersection(nodes_to_rm)
            num_of_ancestors = len(ancestors_n)
            if num_of_ancestors > 1:
                # add edge
                ancestors_dict = dict([(i, node_df.at[i, 'year']) for i in ancestors_n])
            #    ancestors_dict = sorted(ancestors_dict.items() ,  key=lambda x: x[1])
                for u_dict,v_dict in combinations(ancestors_dict.items(),2):
                    if not prev_G.has_edge(u_dict[0], v_dict[0]) and not prev_G.has_edge(u_dict[0], v_dict[0]):
                        if not G.has_edge(u_dict[0], v_dict[0]) and not G.has_edge(v_dict[0], u_dict[0]): # 取第一条边
                            if u_dict[1] < v_dict[1]: 
                                G.add_edge(u_dict[0], v_dict[0], weight = 2*wgt_shared/num_of_ancestors/(num_of_ancestors-1))
                            elif u_dict[1] > v_dict[1]:
                                G.add_edge(v_dict[0], u_dict[0], weight = 2*wgt_shared/num_of_ancestors/(num_of_ancestors-1))   
                            else: # paper on same year, bidirectional edges
                                G.add_edge(u_dict[0], v_dict[0], weight = wgt_shared/num_of_ancestors/(num_of_ancestors-1))
                                G.add_edge(v_dict[0], u_dict[0], weight = wgt_shared/num_of_ancestors/(num_of_ancestors-1))
                            new_edge_wgt_sum += 2*wgt_shared/num_of_ancestors/(num_of_ancestors-1)    
                G.remove_node(n)
            
     # get entropy
    E = 0
    for c in sorted(nx.strongly_connected_components(G), key=len, reverse=True):
        sc_node_num = len(c)
        if sc_node_num>1:
            sub_G = G.subgraph(c).copy()
            # get res
            res = sum([wgt**2 * sub_G.in_degree(u)/sub_G.in_degree(v)/sub_G.out_degree(u)**2 \
                       for (u,v,wgt) in sub_G.edges.data('weight')])  #combinations(c, 2):   
            E += 1-1/sc_node_num - res/2/sc_node_num**2
            for u,v in combinations(sub_G.nodes,2):
                if sub_G.has_edge(u,v) and sub_G.has_edge(v,u):
                    E -= sub_G[u][v]['weight']*sub_G[v][u]['weight']/sub_G.in_degree(v)/sub_G.out_degree(u)
        else:
            # strongly connected component containing 1 node
            # von neumann entropy = 0
            break
    return E, new_edge_wgt_sum

## read input

In [4]:
ref_df = pd.read_csv(r'99188113domain_reference.csv')
ref_df.head()

,paper_id,reference_id,year
0,8777697,99188113,1995
1,498094871,99188113,1995
2,352699142,99188113,1995
3,39453343,99188113,1995
4,470391866,99188113,1995


In [5]:
paper_df = pd.read_csv(r'99188113domain_paper.csv')
paper_df.head()

,paper_id,year
0,352699142,1993
1,47465484,1993
2,34702267,1993
3,39453343,1994
4,498094871,1994


## run single-domain T evolution

In [4]:
def getEvolvingTemperature(ref_df, paper_df, lead_paper_id, evolve_step=1):
    '''
    Compute evolving node temperature based on mainly domain skeleton tree and node tree entropy
    Visualise every evolve_step-year
    Input: 
        ref_df: pandas Dataframe, columns = ['paper_id','reference_id','year']
        paper_df:  pandas Dataframe, columns = ['paper_id','year']
        lead_paper_id: int, original id of the pionneering work
        evolve_step: interval between 2 visualization, unit: year 
    Output:
        None
    '''
    domain = Domain()
    start_year = ref_df['year'].min()
    end_year = ref_df['year'].max()
    T = int((end_year - start_year)/evolve_step)
    delta = (end_year - start_year)%evolve_step 
    domain.setBirthYear(start_year)
    
    # record network size and average temperature for visualization
    yr_stamps = list()
    yr_node_number = list()
    yr_edge_number = list()
    yr_mass = list()
    yr_volume = list()
    yr_avg_growth_temperatures = list()
    
    von_neumann_entropies = list()
    von_neumann_entropies_shrink = list()
    yr_avg_struct_temperatures = list()
    
    is_big_graph = True if paper_df.shape[0]>5000 else False
    
    for t in range(T+1):
        
        print('round',t)
        t3 = perf_counter()
        
        prev_G = deepcopy(domain.G)
        
        t1 = perf_counter()
        domain.evolve(ref_df, lead_paper_id, start_year+delta+t*evolve_step, evolve_step)
        print('Evolve domain elapsed time: {}s'.format(perf_counter() - t1))
        
        if t>=1:
            # get equivalent shrinked network, remove all new nodes, add edges between old nodes
            curr_G = deepcopy(domain.G)
            H, delta_edge_sum = get_von_neumann_E_shrinked(prev_G, curr_G, paper_df)
            H_prev = von_neumann_entropies[-1]
            try:
                struct_Temperature = delta_edge_sum/(H - H_prev) if delta_edge_sum else 0
            except: # float division by 0 error
                struct_Temperature = 10*delta_edge_sum
            avg_struct_T = struct_Temperature/domain.num_of_nodes 
            domain.setStructTemperature(avg_struct_T)
            print('old E: {o}, new_E: {n}, struct_T: {T}'.format(o=H_prev,n=H,T= avg_struct_T))
    
            von_neumann_entropies_shrink.append(H)
            yr_avg_struct_temperatures.append(avg_struct_T)
        else:
            # first round, no struct change
            yr_avg_struct_temperatures.append(0)
            domain.setStructTemperature(0)
            
        domain.getVonNeumannEntropy()
        von_neumann_entropies.append(domain.von_neumann_entropy)
        
        
        t1 = perf_counter()
        domain.setNodeReductionIdx()
        print('Get node reduction index elapsed time: {}s'.format(perf_counter() - t1))
        
       
        t1 = perf_counter()
        domain.buildSkeletonTree()
        print("Build skeleton tree elapsed time: {}s".format(perf_counter() - t1))
        
        
        ## compute tree entropy
        t1 = perf_counter()    
        domain.setNodeTreeEntropy()
        print("Compute tree entropy elapsed time: {}s".format(perf_counter() - t1))
 
  
        ## compute domain average temperature and node temperature
        t1 = perf_counter()
        if t == 0:    
            domain.getAvgTemperature(is_initial = True, is_big_graph = is_big_graph)
            domain.spreadTemperature(former_population = 0)
        else:    
            domain.getAvgTemperature(is_initial = False, is_big_graph = is_big_graph,
                                                         old_avg_temperature = yr_avg_growth_temperatures[-1],
                                                         old_V = yr_volume[-1],
                                                         old_N = yr_mass[-1])
            domain.spreadTemperature(former_population = yr_node_number[-1])
        print("Compute average and node temperature elapsed time: {}s".format(perf_counter() - t1))
          
        
        # export node info
        df_node_export = pd.DataFrame({'id':list(domain.G.nodes),'T': domain.node_temperature})       
        df_node_export['tree_entropy'] = domain.node_tree_entropy
        df_node_export.to_csv(str(lead_paper_id)+'node_'+ str(domain.evolving_until_yr)+'.csv', index = False)
        # export edge info
        df_edge_export = pd.DataFrame(list(domain.skeleton_tree.edges))
        df_edge_export.columns = ['reference_id','paper_id']
        df_edge_export.to_csv(str(lead_paper_id)+'edge_'+ str(domain.evolving_until_yr)+'.csv', index = False)
        
        
        t4 = perf_counter()
        print('total running time: {}s'.format(t4-t3))
        print()
        
        yr_stamps.append(domain.evolving_until_yr)
        yr_node_number.append(domain.num_of_nodes)
        yr_edge_number.append(domain.num_of_edges)  
        yr_mass.append(domain.mass)
        yr_volume.append(domain.volume)
        yr_avg_growth_temperatures.append(domain.average_growth_temperature)

        
    d = {'year':yr_stamps, 'node number':yr_node_number,
         'edge number':yr_edge_number, 'mass':yr_mass, 'volume': yr_volume,
         'T_growth_t': yr_avg_growth_temperatures,'T_struct_t': abs(np.array(yr_avg_struct_temperatures)),
         'T_t':np.array(yr_avg_growth_temperatures)+abs(np.array(yr_avg_struct_temperatures))}    
    df_network_evolve_info = pd.DataFrame(data=d)
    with pd.ExcelWriter(str(lead_paper_id)+' domain stats.xlsx') as writer:
        df_network_evolve_info.to_excel(writer)

In [7]:
pioneer_id = 99188113
getEvolvingTemperature(ref_df, paper_df, pioneer_id, evolve_step = 3)

round 0
Evolve domain elapsed time: 0.00860419267947205s
von neumann entropy
2.291361239711934
Get node reduction index elapsed time: 2.2236642273839777s
Build skeleton tree elapsed time: 0.8759111166902862s
Compute tree entropy elapsed time: 0.20541950243710794s
volume:586, mass:494.41769774908636, T_growth:2.5829223433433177
Compute average and node temperature elapsed time: 0.035238014304236565s
total running time: 3.3738805751737604s

round 1
Evolve domain elapsed time: 0.0159661177957382s
old E: 2.291361239711934, new_E: -4.415954206355881, struct_T: -0.008222634842834385


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide


von neumann entropy
8.763838737119505
Get node reduction index elapsed time: 51.708900355190494s
Build skeleton tree elapsed time: 12.499493819163497s
Compute tree entropy elapsed time: 2.77261355393955s
volume:2235, mass:1779.6617842431435, T_growth:2.7368298243623146


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide


Compute average and node temperature elapsed time: 0.3258859575104225s
total running time: 67.48784972053387s

round 2
Evolve domain elapsed time: 0.03377007224756312s
old E: 8.763838737119505, new_E: 7.2155948014382965, struct_T: -0.08949668063093381
von neumann entropy
16.418897134560595
Get node reduction index elapsed time: 346.6668504973478s
Build skeleton tree elapsed time: 64.14653538523163s
Compute tree entropy elapsed time: 12.390054031410386s
volume:4761, mass:3564.2200526686192, T_growth:2.910994690510742


...

Compute average and node temperature elapsed time: 30.13359295303235s
total running time: 12559.760697588068s

round 7
Evolve domain elapsed time: 0.2671897848449589s
old E: 28.26054933363467, new_E: -16.48999816017973, struct_T: -0.0010508305097180152
von neumann entropy
28.76054933363467
Get node reduction index elapsed time: 14237.12583327662s
Build skeleton tree elapsed time: 1116.6680800707181s
Compute tree entropy elapsed time: 200.4992268885544s
volume:16927, mass:11352.527141339822, T_growth:3.2493402213721305


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide


Compute average and node temperature elapsed time: 42.23266546613013s
total running time: 15602.421632592755s

round 8
Evolve domain elapsed time: 0.34853345077863196s
old E: 28.76054933363467, new_E: -56.13302849005679, struct_T: -5.531978134770794e-05
von neumann entropy
28.76054933363467
Get node reduction index elapsed time: 14346.428840071887s
Build skeleton tree elapsed time: 1117.3819939258246s
Compute tree entropy elapsed time: 219.89368509875203s
volume:17046, mass:11431.177148790819, T_growth:3.2496700673032475


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide


Compute average and node temperature elapsed time: 41.69318706335616s
total running time: 15733.072351947834s



## domain group

In [5]:
class DomainGroup(Domain):
    '''
    a group of similar domains with one pionneering paper
    '''
    def __init__(self, num, T):
        self.domain_number = 0
        self.domains = list()
        self.domain_ages = np.zeros(num, dtype = int)

        self.domain_avg_temperatures = np.zeros([T, num])
        self.domains_helped = list() # -1: no domain to help 
        self.total_rounds = T
        self.current_round = 0
    
    def initializeDomains(self, start_yrs):
        # create domains
        for yr in start_yrs:
            d = Domain()
            d.setBirthYear(yr)
            self.domains.append(d)
        
    def setDomainAvgTemperatures(self, t, domain_idx, is_born = True):
        if not is_born or self.domains[domain_idx].evolving_until_yr == self.domains[domain_idx].birth_yr:
            self.domain_avg_temperatures[t, domain_idx] = -1
        else:
            self.domain_avg_temperatures[t, domain_idx] = self.domains[domain_idx].average_temperature
        
    def updateDomainAges(self):
        self.domain_ages = np.array([d.evolving_until_yr - d.birth_yr for d in self.domains])
        self.domain_ages = np.where(self.domain_ages < 0, 0, self.domain_ages)
    
    def decideHelpWho(self):
        '''
        identify the domain with the slowest growth during the last round
        Input:
            None
        Output:
            int, index of the slowest-growing domain in self.domains
        '''
        recent_avg_temperature_growth = np.true_divide(self.domain_avg_temperatures[self.current_round,:],\
                                                       self.domain_avg_temperatures[self.current_round-1,:]) 
        print("Decide help who")
        print(self.domain_avg_temperatures[self.current_round,:])
        print(self.domain_avg_temperatures[self.current_round-1,:])
        print(recent_avg_temperature_growth)
        
        if min(recent_avg_temperature_growth)>1:
             # both/all grow, no need to help
            return np.array([])
        else:
            return np.argwhere(abs(recent_avg_temperature_growth)<1).flatten()
        
   
    def adjustAvgTemp(self, target_domain_idx, t):
        '''
        adjust the average temperature of the slowest-growing domain
        Input: 
            target_domain_idx: index of the slowest-growing domain in self.domains
        Output:
            unscaled avg temperature after adjustment
        '''
        # get domain average temperatures before average temperature rescaling        
        age_sum = np.sum(self.domain_ages)
        
        print("adjustAvgTemp")
        print("domain_to_help", target_domain_idx)
        print("current avg_T")
        print(self.domain_avg_temperatures[t])
        
        # all those with positive temperature increases help the rest
        energy_to_spare = 0
        for idx, d in enumerate(self.domains):
            if idx not in target_domain_idx:
                energy_to_spare += d.mass/(1+age_sum)*self.domain_avg_temperatures[t][idx]
        return energy_to_spare        
   
    def updateDomainInHelp(self, target_domain_idx, t, delta_U):

        delta_T = delta_U/sum([self.domains[d].mass for d in target_domain_idx])
        
        for i, domain in enumerate(self.domains):
            if i in target_domain_idx:
                new_avg_T = domain.average_temperature + delta_T
            else:
                new_avg_T = domain.average_temperature * sum(self.domain_ages)/(1+sum(self.domain_ages))
            domain.node_temperature *= (new_avg_T/domain.average_temperature) 
            domain.average_struct_temperature *= (new_avg_T/domain.average_temperature)
            domain.average_growth_temperature *= (new_avg_T/domain.average_temperature)
            domain.average_temperature = new_avg_T
       
        
    
    def updateDomainGroupRecord(self, target_domain_idx = None, t = None):
        self.current_round += 1
        self.domain_number = len(np.nonzero(self.domain_ages)[0])
        if target_domain_idx:
            for i, domain in enumerate(self.domains):
                self.domain_avg_temperatures[t, i] = domain.average_temperature if domain.average_temperature>0 else -1
            self.domains_helped.append(target_domain_idx)
            print("updateDomainGroupRecord")
            print(self.domain_avg_temperatures)
        

## run multiple domains T evolution

In [6]:
def getEvolvingTemperatureForest(df_ref_list, df_paper_list, lead_paper_ids, evolve_step=1):
    '''
    Compute evolving node temperature based on domain skeleton tree, node tree entropy and forest helping mechanism
    Visualise every evolve_step-year
    Input: 
        df_ref_list: pandas Dataframe, columns = ['paper_id','reference_id','year'], each df is a domain
        df_paper_list: pandas Dataframe, columns = ['paper_id','year'], each df is a domain
        lead_paper_ids: list of int, original ids of the pionneering work
        evolve_step: interval between 2 visualization, unit: year 
    Output:
        None
    '''
    # initialize domain group
    N = len(df_ref_list)
    start_years = [df['year'].min() for df in df_ref_list]
    end_years = [df['year'].max() for df in df_ref_list]
    domain_rank = np.argsort(start_years)
    ordered_df_ref_list = list()
    ordered_df_paper_list = list()
    ordered_lead_paper_ids = list()
    for i in range(N):
        old_idx = np.argwhere(domain_rank == i).flatten()[0]
        ordered_df_ref_list.append(df_ref_list[old_idx])
        ordered_df_paper_list.append(df_paper_list[old_idx])
        ordered_lead_paper_ids.append(lead_paper_ids[old_idx])
    T = int((max(end_years) - min(start_years))/evolve_step)
    delta = (max(end_years) - min(start_years))%evolve_step  # the beginning period where there's only one domain 
    
    # initialize domains
    domain_group = DomainGroup(N, T+1)
    domain_group.initializeDomains(sorted(start_years))
    
    # judge if big graph, for T_growth_t
    are_big_graphs = list([True if df.shape[0]>5000 else False for df in ordered_df_paper_list])
    
    # record network size and average temperature for visualization
    yr_stamps = list()
    yr_node_number = np.zeros([T+1,N], dtype = int)
    yr_edge_number = np.zeros([T+1,N], dtype = int)
    yr_mass =  np.zeros([T+1,N])
    yr_volume = np.zeros([T+1,N])
    von_neumann_entropies = np.zeros([T+1,N])
    von_neumann_entropies_shrink = np.zeros([T,N])
    yr_avg_temperature = np.zeros([T+1,N])
    
    current_yr = min(start_years)+delta
    
    prev_avg_growth_temperature = np.zeros(N)
    
    for t in range(T+1):
        
        print('round',t)
        t3 = perf_counter()
        
        print('Current year', current_yr)
        
        t1 = perf_counter()
        for domain_idx, (domain, lead_paper_id, df, paper_df,is_big_graph) in enumerate(zip(domain_group.domains,
                                                                                   ordered_lead_paper_ids, 
                                                                                   ordered_df_ref_list,
                                                                                   ordered_df_paper_list,
                                                                                   are_big_graphs)):
            
            if domain.birth_yr >= current_yr+evolve_step:
                # not born by the end of this round
                domain_group.setDomainAvgTemperatures(t, domain_idx, is_born = False)
                continue    
            
            prev_G = deepcopy(domain.G)
                
            domain.evolve(df, lead_paper_id, current_yr, evolve_step)
            
            if prev_G.number_of_nodes():
                # get equivalent shrinked network, remove all new nodes, add edges between old nodes
                curr_G = deepcopy(domain.G)
                H, delta_edge_sum = get_von_neumann_E_shrinked(prev_G, curr_G, paper_df)
                H_prev = von_neumann_entropies[t-1, domain_idx]
                try:
                    struct_Temperature = delta_edge_sum/(H - H_prev) if delta_edge_sum else 0
                except: # float division by 0 error
                    struct_Temperature = 10*delta_edge_sum  
                avg_struct_T = struct_Temperature/domain.num_of_nodes 
                domain.setStructTemperature(avg_struct_T)
                von_neumann_entropies_shrink[t-1, domain_idx] = H
            else:
                # first round, no struct change
                domain.setStructTemperature(0)
            
            domain.getVonNeumannEntropy()
            von_neumann_entropies[t, domain_idx] = domain.von_neumann_entropy
       
            domain.setNodeReductionIdx()
          
            domain.buildSkeletonTree()
        
            domain.setNodeTreeEntropy()

            ## compute domain average temperature and node temperature
            if prev_G.number_of_nodes() == 0:    
                domain.getAvgTemperature(is_initial = True, is_big_graph = is_big_graph)
                domain.spreadTemperature(former_population = 0)
            else:    
                domain.getAvgTemperature(is_initial = False, is_big_graph = is_big_graph,
                                         old_avg_temperature = prev_avg_growth_temperature[domain_idx],
                                         old_V = yr_volume[t-1, domain_idx],
                                         old_N = yr_mass[t-1, domain_idx])
                domain.spreadTemperature(former_population = yr_node_number[t-1, domain_idx])
            
            # update group avg temperature
            domain_group.setDomainAvgTemperatures(t, domain_idx)
                
        print("Evolve domain group: {}s".format(perf_counter() - t1))
        
        
        print("avg_T before adjustment: ")    
        print([d.average_temperature for d in domain_group.domains])
        
        print("avg_struct_T:")
        print([d.average_struct_temperature for d in domain_group.domains])
        
        domain_group.updateDomainAges()
        print("domain ages:", domain_group.domain_ages)
        
        if domain_group.domain_number >= 2:
            # forest helping mechanism
            t1 = perf_counter()
            domain_to_help = domain_group.decideHelpWho()
            print("domain_to_help", domain_to_help)
            if len(list(domain_to_help)):
                energy_to_share = domain_group.adjustAvgTemp(domain_to_help, t) 
                domain_group.updateDomainInHelp(domain_to_help, t, energy_to_share)
            domain_group.updateDomainGroupRecord(list(domain_to_help), t)
            print("Forest helping: {}s".format(perf_counter() - t1))
        else:
            domain_group.updateDomainGroupRecord()
            
        # output domain group evolution after forest helping  
        for i, (domain, lead_paper_id) in enumerate(zip(domain_group.domains, ordered_lead_paper_ids)):
#             # export node info
#             df_node_export = pd.DataFrame({'id':list(domain.G.nodes),'T': domain.node_temperature})       
#             df_node_export['tree_entropy'] = domain.node_tree_entropy
#             df_node_export.to_csv(str(lead_paper_id)+'node_'+str(t)+'_'+ str(domain.evolving_until_yr)+'.csv', index = False)
#             # export edge info
#             df_edge_export = pd.DataFrame(list(domain.skeleton_tree.edges))
#             df_edge_export.columns = ['reference_id','paper_id']
#             df_edge_export.to_csv(str(lead_paper_id)+'edge_'+str(t)+'_'+ str(domain.evolving_until_yr)+'.csv', index = False)
            
            yr_node_number[t, i] = domain.num_of_nodes
            yr_edge_number[t, i] = domain.num_of_edges
            yr_mass[t, i] = domain.mass
            yr_volume[t, i] = domain.volume
            yr_avg_temperature[t, i] = domain.average_temperature
            prev_avg_growth_temperature[i] = domain.average_growth_temperature 
        
        yr_stamps.append(current_yr+evolve_step-1)
        current_yr += evolve_step
        
        t4 = perf_counter()
        print('total running time: {}s'.format(t4-t3))
        print()
        
    
    # output stats for each domain
    for i, (domain, pioneer_id) in enumerate(zip(domain_group.domains, ordered_lead_paper_ids)):
        d = {'year':yr_stamps, 'node number':yr_node_number[:,i],
             'edge number':yr_edge_number[:,i], 'mass':yr_mass[:,i], 'volume': yr_volume[:,i],
             'T_t forest':yr_avg_temperature[:,i]}    
        df_network_evolve_info = pd.DataFrame(data=d)
        with pd.ExcelWriter(str(pioneer_id)+' domain stats.xlsx') as writer:
            df_network_evolve_info.to_excel(writer)

In [9]:
# wireless networks
# ref_df1 = pd.read_csv(r'wireless networks forest/62270017domain_reference.csv')
# ref_df2 = pd.read_csv(r'wireless networks forest/438420345domain_reference.csv')
# paper_df1 = pd.read_csv(r'wireless networks forest/62270017domain_paper.csv')
# paper_df2 = pd.read_csv(r'wireless networks forest/438420345domain_paper.csv')

# GRU LSTM
ref_df1 = pd.read_csv(r'RNN memory layer forest/168338164domain_reference.csv')
ref_df2 = pd.read_csv(r'RNN memory layer forest/56158074domain_reference.csv')
paper_df1 = pd.read_csv(r'RNN memory layer forest/168338164domain_paper.csv')
paper_df2 = pd.read_csv(r'RNN memory layer forest/56158074domain_paper.csv')

# RNN word embeddings
# ref_df1 = pd.read_csv(r'word embedding forest/223688399domain_reference.csv')
# ref_df2 = pd.read_csv(r'word embedding forest/256500874domain_reference.csv')
# ref_df3 = pd.read_csv(r'word embedding forest/372720438domain_reference.csv')
# paper_df1 = pd.read_csv(r'word embedding forest/223688399domain_paper.csv')
# paper_df2 = pd.read_csv(r'word embedding forest/256500874domain_paper.csv')
# paper_df3 = pd.read_csv(r'word embedding forest/372720438domain_paper.csv')

# df_ref_list = [ref_df1, ref_df2, ref_df3]
# df_paper_list = [paper_df1, paper_df2, paper_df3]
# lead_paper_ids = [223688399, 256500874, 372720438]
df_ref_list = [ref_df1, ref_df2]
df_paper_list = [paper_df1, paper_df2]
lead_paper_ids = [168338164, 56158074]
getEvolvingTemperatureForest(df_ref_list, df_paper_list, lead_paper_ids, evolve_step=2)

round 0
Current year 1998
von neumann entropy
0
volume:17, mass:14.333333333333332, T_growth:2.7473041750434675
Evolve domain group: 0.020964462601114064s
avg_T before adjustment: 
[2.7473041750434675, 0]
avg_struct_T:
[0, 0]
domain ages: [2 0]
total running time: 0.022342840580677148s

round 1
Current year 2000
von neumann entropy
0.25
volume:56, mass:37.4404703686275, T_growth:3.464589273381462
Evolve domain group: 0.054918259964324534s
avg_T before adjustment: 
[inf, 0]
avg_struct_T:
[inf, 0]
domain ages: [4 0]
total running time: 0.05551418188406387s

round 2
Current year 2002
von neumann entropy
0.25


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide
c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: divide by zero encountered in double_scalars
c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide
c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:388: RuntimeWarning: invalid value encountered in multiply


volume:102, mass:62.77337411892641, T_growth:3.763827615927091
Evolve domain group: 0.13890023138810648s
avg_T before adjustment: 
[3.86393870877533, 0]
avg_struct_T:
[0.10011109284823903, 0]
domain ages: [6 0]
total running time: 0.14021831194258993s

round 3
Current year 2004
von neumann entropy
1.5455555555555556


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide


volume:156, mass:95.06520149887511, T_growth:3.8010891092095727
Evolve domain group: 0.37413988015032373s
avg_T before adjustment: 
[3.944057878986084, 0]
avg_struct_T:
[0.142968769776511, 0]
domain ages: [8 0]
total running time: 0.37523122828133637s

round 4
Current year 2006
von neumann entropy
2.0455555555555556


...

von neumann entropy
32.766257589467166
volume:16777, mass:6610.636385225673, T_growth:5.878629104785079


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide


von neumann entropy
2.3055555555555554
volume:2282, mass:1576.6176732051877, T_growth:27.15077697134515


c:\users\ldr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:371: RuntimeWarning: divide by zero encountered in true_divide


Evolve domain group: 19879.774643839686s
avg_T before adjustment: 
[5.953613119969601, 27.16182623303536]
avg_struct_T:
[0.0749840151845222, 0.011049261690210359]
domain ages: [23  5]
Decide help who
[ 5.95361312 27.16182623]
[ 5.92178905 27.3796276 ]
[1.00537406 0.99204513]
domain_to_help [1]
adjustAvgTemp
domain_to_help [1]
current avg_T
[ 5.95361312 27.16182623]
updateDomainGroupRecord
[[ 2.74730418 -1.        ]
 [        inf -1.        ]
 [ 3.86393871 -1.        ]
 [ 3.94405788 -1.        ]
 [ 3.93924767 -1.        ]
 [ 3.97411392 -1.        ]
 [ 4.12037991 -1.        ]
 [ 4.15621191 -1.        ]
 [ 4.58854574 -1.        ]
 [ 4.31633652 25.87988961]
 [ 5.92178905 27.3796276 ]
 [ 5.74831612 28.02262075]]
Forest helping: 0.005970502912532538s
total running time: 19879.783884155593s

